In [18]:
import requests
import pandas as pd
#import json
#import numpy as np

In [19]:
from datetime import date

today = date.today()
today_date = today.strftime("%Y-%m-%d")
today_date

'2022-10-19'

--- here starts currency collection from ecb-------

In [20]:
# Building blocks for the URL
entrypoint = 'https://sdw-wsrest.ecb.europa.eu/service/' # Using protocol 'https'
resource = 'data'           # The resource for data queries is always'data'
flowRef ='EXR'              # Dataflow describing the data that needs to be returned, exchange rates in this case
key = 'D.SEK.EUR.SP00.A'    # Defining the dimension values, explained below

# Define the parameters
parameters = {
    'startPeriod': '2022-01-01',  # Start date of the time series
    'endPeriod': today_date,     # End of the time series
    #'format':'jsondata'
}

In [21]:
# Construct the URL: https://sdw-wsrest.ecb.europa.eu/service/data/EXR/D.CHF.EUR.SP00.A
request_url = entrypoint + resource + '/'+ flowRef + '/' + key

# Make the HTTP request again, now requesting for CSV format
response = requests.get(request_url, params=parameters, headers={'Accept': 'text/csv'})
#response = requests.get(request_url, params=parameters, headers={'Accept': 'application/vnd.sdmx.data+json;version=1.0.0-wd'})
# Response succesful? (Response code 200)
#print(response)

# Construct the URL: https://sdw-wsrest.ecb.europa.eu/service/data/EXR/D.CHF.EUR.SP00.A
request_url = entrypoint + resource + '/'+ flowRef + '/' + key

# Make the HTTP request again, now requesting for CSV format
response = requests.get(request_url, params=parameters, headers={'Accept': 'application/vnd.sdmx.data+json;version=1.0.0-wd'})
# Response succesful? (Response code 200)
response.json()


In [22]:
open('csvs/temp.csv', 'wb').write(response.content)
currency_df = pd.read_csv('csvs/temp.csv')

In [23]:
#currency_df.keys()

In [24]:
currency_df.tail(1)


,KEY,FREQ,CURRENCY,CURRENCY_DENOM,EXR_TYPE,EXR_SUFFIX,TIME_PERIOD,OBS_VALUE,OBS_STATUS,OBS_CONF,...,COMPILATION,COVERAGE,DECIMALS,NAT_TITLE,SOURCE_AGENCY,SOURCE_PUB,TITLE,TITLE_COMPL,UNIT,UNIT_MULT
205,EXR.D.SEK.EUR.SP00.A,D,SEK,EUR,SP00,A,2022-10-19,10.9448,A,F,...,NaN,NaN,4,NaN,4F0,NaN,Swedish krona/Euro,"ECB reference exchange rate, Swedish krona/Eur...",SEK,0


In [29]:
ts = currency_df.filter(['TIME_PERIOD', 'OBS_VALUE', 'KEY', 'CURRENCY'], axis=1)
# 'TIME_PERIOD' was of type 'object' (as seen in df.info). Convert it to datetime first
ts['TIME_PERIOD'] = pd.to_datetime(ts['TIME_PERIOD'])
# Set 'TIME_PERIOD' to be the index
ts = ts.set_index('TIME_PERIOD')
# Print the last 5 rows to screen
ts.tail(5)

,OBS_VALUE,KEY,CURRENCY
TIME_PERIOD,,,
2022-10-13,11.0098,EXR.D.SEK.EUR.SP00.A,SEK
2022-10-14,11.0035,EXR.D.SEK.EUR.SP00.A,SEK
2022-10-17,10.9893,EXR.D.SEK.EUR.SP00.A,SEK
2022-10-18,10.9060,EXR.D.SEK.EUR.SP00.A,SEK
2022-10-19,10.9448,EXR.D.SEK.EUR.SP00.A,SEK


In [27]:
rate_sek = currency_df.tail(1)['OBS_VALUE'].values[0]
rate_sek

10.9448

--- end of currency collection -----